## Task 1

In [2]:
# Necessary installs
!pip install pyttsx3
!pip install numpy
!pip install opencv-python
!pip install pypiwin32


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.


### Detect cars on the 'Main Street'.

In [1]:
import cv2

# create a VideoCapture object and read the frames from an input file
# ensure path to Traffic_Lamarie_1.mp4 is correct
video=cv2.VideoCapture('Traffic_Laramie_1.mp4')

# check if the video opened successfully
if (video.isOpened()== False): 
  print("Error opening video file")

# initialise initial frame as none
initial_frame = None

# we start an infinite loop and keep reading frames from the video until we press 'q'
while True:
    check, frame = video.read()
    if check == True:
        # gray conversion and noise reduction (smoothening)
        gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        blur_frame=cv2.GaussianBlur(gray_frame,(25,25),0)

        # the first captured frame is the baseline image
        if initial_frame is None:
            initial_frame = blur_frame
            continue

        # the difference between the baseline and the new frame
        delta_frame=cv2.absdiff(initial_frame,blur_frame)

        # the difference (the delta_frame) is converted into a binary image
        # if a particular pixel value is greater than a certain threshold (specified by us here as 18),
        # it will be assigned the value for White (255) else Black(0)
        # threshold_frame=cv2.threshold(delta_frame,18,255, cv2.THRESH_BINARY)[1]

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))  # kernel to apply to the morphology
        closing = cv2.morphologyEx(delta_frame, cv2.MORPH_CLOSE, kernel)
        opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
        dilation = cv2.dilate(opening, kernel)
        retvalbin = cv2.threshold(dilation, 12, 255, cv2.THRESH_BINARY)[1]  # removes the shadows    

        # the cv2.findContours() method we will identify all the contours in our image
        (contours,_)=cv2.findContours(retvalbin,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        
        # get height of video frame
        height = 0
        if video.isOpened():
            height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)

        # loop over contours and paint them onto the original video image
        for c in contours:
            # contourArea() method filters out any small contours
            if cv2.contourArea(c) < 2500:
                continue
            (x, y, w, h)=cv2.boundingRect(c)
            # if car on main street, roughly bottom half of the video frame, paint contour
            if y > height / 2:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 1)

        # to better understand the application, we can visualise the different frames generated
        cv2.imshow('Video', frame)
        # cv2.imshow('Baseline image', initial_frame)
        # cv2.imshow('Gray Frame',gray_frame)
        # cv2.imshow('Delta frame', delta_frame)
        # cv2.imshow('Threshold frame', threshold_frame)

        # cv2.imshow('closing', closing)
        # cv2.imshow('opening', opening)
        # cv2.imshow('dilation', dilation)
        # cv2.imshow('retvalbin', retvalbin)
    
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    
    else:
        break

# after the loop release the video object
video.release()

# destroy all the windows
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

-1